# Example code for search of best activation function
### [----->ML From Scratch](https://mlfromscratch.com)
We are going to run a small experiment, using mostly Keras and Scikit-Learn. The goal is to take the MNIST dataset and draw a graph of activations functions with loss over time.

The outline for this notebook is as follows:
1. [Importing packages and small tweaks to MNIST dataset](#1.-Importing-and-preprocessing)
2. [Readying functions, hyperparameters etc.](#2.-Readying-functions-and-hyperparameters)
3. [Fitting data](#3.-Fitting-the-data-with-multiple-activation-functions)
4. [Plotting](#4.-Graph-the-results)

# 1. Importing and preprocessing

In [59]:
# IMPORTS
import tensorflow as tf
import numpy as np
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, LeakyReLU
from keras.layers.noise import AlphaDropout
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K

# LOAD DATA
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# PREPROCESSING
def preprocess_mnist(x_train, y_train, x_test, y_test):
    # Normalizing all images of 28x28 pixels
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)
    
    # Float values for division
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    # Normalizing the RGB codes by dividing it to the max RGB value
    x_train /= 255
    x_test /= 255
    
    # Categorical y values
    y_train = to_categorical(y_train)
    y_test= to_categorical(y_test)
    
    return x_train, y_train, x_test, y_test, input_shape
    
x_train, y_train, x_test, y_test, input_shape = preprocess_mnist(x_train, y_train, x_test, y_test)

# 2. Readying functions and hyperparameters

In [64]:
# Readying neural network model
def build_cnn(activation,
              dropout_rate,
              optimizer):
    model = Sequential()
    
    if(activation == 'selu'):
        model.add(Conv2D(32, kernel_size=(3, 3),
                  activation=activation,
                  input_shape=input_shape,
                  kernel_initializer='lecun_normal'))
        model.add(Conv2D(64, (3, 3), activation=activation, 
                         kernel_initializer='lecun_normal'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(AlphaDropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation=activation, 
                        kernel_initializer='lecun_normal'))
        model.add(AlphaDropout(0.5))
        model.add(Dense(10, activation='softmax'))
    else:
        model.add(Conv2D(32, kernel_size=(3, 3),
                  activation=activation,
                  input_shape=input_shape))
        model.add(Conv2D(64, (3, 3), activation=activation))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation=activation))
        model.add(Dropout(0.5))
        model.add(Dense(10, activation='softmax'))
    
    model.compile(
        loss='binary_crossentropy', 
        optimizer=optimizer, 
        metrics=['accuracy']
    )
    
    return model

# Add the GELU function to Keras
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

# Add leaky-relu so we can use it as a string
get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

act_func = ['selu', 'sigmoid', 'relu', 'elu', 'leaky-relu', 'selu', 'gelu']

/anaconda3/envs/nestedcv_env/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


# 3. Fitting the data with multiple activation functions

In [65]:
result = []

for activation in act_func:
    print('Training with -->{0}<-- activation function\n'.format(activation))
    
    model = build_cnn(activation=activation,
                      dropout_rate=0.2,
                      optimizer='Adam')
    
    history = model.fit(x_train, y_train,
          validation_split=0.33,
          batch_size=128, # 128 is faster, but less accurate. 16/32 recommended for real work
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test))
    
    result.append([history,activation])
    
    K.clear_session()
    del model

print(result)

Training with -->selu<-- activation function

running selu lol
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 144s 2ms/step - loss: 0.0513 - acc: 0.9825 - val_loss: 0.0218 - val_acc: 0.9962
Training with -->sigmoid<-- activation function

no selu 4 u
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
 2944/60000 [>.............................] - ETA: 1:38 - loss: 0.3894 - acc: 0.8956

KeyboardInterrupt: 

# 4. Graph the results